In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns             # statistical data visualization


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/electricity-theft-detection/visualization.csv
/kaggle/input/electricity-theft-detection/Validation_SansFLAG.csv
/kaggle/input/electricity-theft-detection/Validation.csv
/kaggle/input/electricity-theft-detection/preprocessedR95.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, confusion_matrix, \
    precision_recall_fscore_support, roc_auc_score
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.layers import Dense, Conv1D, Flatten, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

tf.random.set_seed(1234)
epochs_number = 150  # number of epochs for the neural networks
test_set_size = 0.2  # percentage of the test size comparing to the whole dataset
oversampling_flag = 0  # set to 1 to over-sample the minority class
oversampling_percentage = 0.2  # percentage of the minority class after the oversampling comparing to majority class


# Definition of functions
def read_data():
    rawData = pd.read_csv('/kaggle/input/electricity-theft-detection/preprocessedR95.csv')

    # Setting the target and dropping the unnecessary columns
    y = rawData[['FLAG']]
    X = rawData.drop(['FLAG', 'CONS_NO'], axis=1)

    print('Normal Consumers:                    ', y[y['FLAG'] == 0].count()[0])
    print('Consumers with Fraud:                ', y[y['FLAG'] == 1].count()[0])
    print('Total Consumers:                     ', y.shape[0])
    print("Classification assuming no fraud:     %.2f" % (y[y['FLAG'] == 0].count()[0] / y.shape[0] * 100), "%")

    # columns reindexing according to dates
    X.columns = pd.to_datetime(X.columns)
    X = X.reindex(X.columns, axis=1)

    # Splitting the dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y['FLAG'], test_size=test_set_size, random_state=0)
    print("Test set assuming no fraud:           %.2f" % (y_test[y_test == 0].count() / y_test.shape[0] * 100), "%\n")

    # Oversampling of minority class to encounter the imbalanced learning
    if oversampling_flag == 1:
        over = SMOTE(sampling_strategy=oversampling_percentage, random_state=0)
        X_train, y_train = over.fit_resample(X_train, y_train)
        print("Oversampling statistics in training set: ")
        print('Normal Consumers:                    ', y_train[y_train == 0].count())
        print('Consumers with Fraud:                ', y_train[y_train == 1].count())
        print("Total Consumers                      ", X_train.shape[0])

    return X_train, X_test, y_train, y_test


def results(y_test, prediction):
    print("Accuracy", 100 * accuracy_score(y_test, prediction))
    print("RMSE:", mean_squared_error(y_test, prediction, squared=False))
    print("MAE:", mean_absolute_error(y_test, prediction))
    print("F1:", 100 * precision_recall_fscore_support(y_test, prediction)[2])
    print("AUC:", 100 * roc_auc_score(y_test, prediction))
    print("Confusion matrix: \n",confusion_matrix(y_test, prediction), "\n")


def CNN2D(X_train, X_test, y_train, y_test):
    print('2D - Convolutional Neural Network:')

    # Transforming every row of the train set into a 2D array and then into a tensor
    n_array_X_train = X_train.to_numpy()
    n_array_X_train_extended = np.hstack((n_array_X_train, np.zeros(
        (n_array_X_train.shape[0], 2))))  # adding two empty columns in order to make the number of columns
    # an exact multiple of 7
    week = []
    for i in range(n_array_X_train_extended.shape[0]):
        a = np.reshape(n_array_X_train_extended[i], (-1, 7, 1))
        week.append(a)
    X_train_reshaped = np.array(week)

    # Transforming every row of the train set into a 2D array and then into a tensor
    n_array_X_test = X_test.to_numpy()  # X_test to 2D - array
    n_array_X_train_extended = np.hstack((n_array_X_test, np.zeros((n_array_X_test.shape[0], 2))))
    week2 = []
    for i in range(n_array_X_train_extended.shape[0]):
        b = np.reshape(n_array_X_train_extended[i], (-1, 7, 1))
        week2.append(b)
    X_test_reshaped = np.array(week2)

    input_shape = (1, 148, 7, 1)  # input shape of the tensor

    # Model creation
    model = Sequential()
    model.add(Conv2D(kernel_size=(7, 3), filters=32, input_shape=input_shape[1:], activation='relu',
                     data_format='channels_last'))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
    # model.summary()
    #     model.fit(X_train_reshaped, y_train, validation_split=0.1, epochs=i, shuffle=False, verbose=0)
    model.fit(X_train_reshaped, y_train, validation_split=0.1, epochs=epochs_number, shuffle=False, verbose=1)

    # prediction = model.predict_classes(X_test)
    #prediction = model.predict_classes(X_test_reshaped)
    predictions_proba = model.predict(X_test_reshaped)
    predictions = (predictions_proba > 0.5).astype(int)  # Apply threshold of 0.5
    model.summary()
    results(y_test, predictions)

# ----Main----
X_train, X_test, y_train, y_test = read_data()
CNN2D(X_train, X_test, y_train, y_test)

2024-04-16 22:56:24.204315: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 22:56:24.204475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 22:56:24.382648: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_18/1996431349.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('Normal Consumers:                    ', y[y['FLAG'] == 0].count()[0])
/tmp/ipykernel_18/

Normal Consumers:                     35671
Consumers with Fraud:                 2573
Total Consumers:                      38244
Classification assuming no fraud:     93.27 %
Test set assuming no fraud:           93.24 %

2D - Convolutional Neural Network:


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/150
861/861 ━━━━━━━━━━━━━━━━━━━━ 29s 32ms/step - accuracy: 0.9246 - loss: 0.2476 - val_accuracy: 0.9346 - val_loss: 0.2310
Epoch 2/150
861/861 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - accuracy: 0.9337 - loss: 0.2131 - val_accuracy: 0.9382 - val_loss: 0.2041
Epoch 3/150
861/861 ━━━━━━━━━━━━━━━━━━━━ 27s 32ms/step - accuracy: 0.9411 - loss: 0.1792 - val_accuracy: 0.9415 - val_loss: 0.1962
Epoch 4/150
861/861 ━━━━━━━━━━━━━━━━━━━━ 28s 32ms/step - accuracy: 0.9529 - loss: 0.1407 - val_accuracy: 0.9435 - val_loss: 0.1929
Epoch 5/150
861/861 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - accuracy: 0.9656 - loss: 0.1094 - val_accuracy: 0.9418 - val_loss: 0.2195
Epoch 6/150
861/861 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.9730 - loss: 0.0848 - val_accuracy: 0.9350 - val_loss: 0.2366
Epoch 7/150
861/861 ━━━━━━━━━━━━━━━━━━━━ 27s 31ms/step - accuracy: 0.9776 - loss: 0.0697 - val_accuracy: 0.9389 - val_loss: 0.2682
Epoch 8/150
861/861 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.9845 - loss: 0

Model: "sequential"

...

 Total params: 6,868,301 (26.20 MB)

 Trainable params: 2,289,433 (8.73 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,578,868 (17.47 MB)

Accuracy 93.71159628709636
RMSE: 0.2507668979930096
MAE: 0.06288403712903648
F1: [96.68435927 39.1908976 ]
AUC: 64.15606073262306
Confusion matrix: 
 [[7013  119]
 [ 362  155]] 

